In [1]:
import random

from wellytics.models import Form, Question, Metric, Response
import wellytics.api as api

/home/alen/Desktop/Wellytics/wellytics/env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/alen/Desktop/Wellytics/wellytics/env/lib/python3.10/site-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [2]:
_map = lambda f, xs: list(map(f, xs))
_find = lambda f, xs: next(filter(f, xs), None)
_filter = lambda f, xs: list(filter(f, xs))


## Boostrapping

In [3]:
# class Question(BaseModel):
#     id: str
#     createdAt: int
#     updatedAt: int
#     type: str
#     required: bool
#     question: str
#     placeholder: Optional[str]
#     subQuestions: Optional[List[Dict[str, str]]]
#     options: Optional[List[Dict[str, str]]]
#     min: Optional[int]
#     max: Optional[int]

question_template = "What is your favorite color? {i}"

question_n = 10
question_ids = _map(lambda i: f"question-{i}", range(question_n))
question_texts = _map(lambda i: question_template.format(i=i), range(question_n))
question_dicts = [
    {
        "id": id,
        "createdAt": 0,
        "updatedAt": 0,
        "type": "ShortAnswer",
        "required": True,
        "question": question_texts[i],
        "placeholder": None,
        "subQuestions": None,
        "options": None,
        "min": None,
        "max": None,
    }
    for i, id in enumerate(question_ids)
]
questions = _map(lambda d: Question(**d), question_dicts)

In [4]:
# class Form(BaseModel):
#     id: str
#     createdAt: int
#     updatedAt: int
#     active: bool
#     title: str
#     description: str
#     questions: List[str]  # ref to question

form_n = 3
form_ids = _map(lambda i: f"form-{i}", range(form_n))
form_questions_n = 3
form_dicts = [
    {
        "id": id,
        "createdAt": 0,
        "updatedAt": 0,
        "active": True,
        "title": f"Form {i}",
        "description": f"Form {i} description",
        "questions": random.sample(question_ids, form_questions_n),
    }
    for i, id in enumerate(form_ids)
]
forms = _map(lambda d: Form(**d), form_dicts)


In [5]:
tracking_n = 3
tracking_ids = [f"tracking-{i}" for i in range(tracking_n)]


In [6]:
# class Metric(BaseModel):
#     id: str
#     createdAt: int
#     updatedAt: int
#     trackingId: str
#     values: Dict[str, float]

metric_n = 10
metric_values_n = 3
metric_value_ids = [f"metric-value-{i}" for i in range(metric_values_n)]
metric_ids = []
metric_dicts = []

i = 0
for tracking_id in tracking_ids:
    for _ in range(metric_n):
        id = f"metric-{i}"
        metric_ids.append(id)
        metric_dicts.append(
            {
                "id": id,
                "createdAt": 0,
                "updatedAt": 0,
                "trackingId": tracking_id,
                "values": {
                    metric_value_id: random.random()
                    for metric_value_id in metric_value_ids
                },
            }
        )
        i += 1


metrics = _map(lambda d: Metric(**d), metric_dicts)


In [7]:
# class Response(BaseModel):
#     id: str
#     createdAt: int
#     updatedAt: int
#     formId: str
#     trackingId: str
#     answers: Dict[str, Union[str, Dict[str, str]]]
#     metrics: List[str]  # ref to metric

response_n = 10
response_metrics_n = 2
response_dicts = []

i = 0
for form_id in form_ids:
    for _ in range(response_n):
        id = f"response-{i}"
        tracking_id = random.choice(tracking_ids)
        form = _find(lambda f: f.id == form_id, forms)
        _metric_ids = [
            metric_id
            for i, metric_id in enumerate(metric_ids)
            if metrics[i].trackingId == tracking_id
        ]

        response_dict = {
            "id": id,
            "createdAt": 0,
            "updatedAt": 0,
            "formId": form_id,
            "trackingId": random.choice(tracking_ids),
            "answers": {
                question_id: random.choice(["Yes", "No"])
                for question_id in form.questions
            },
            "metrics": random.sample(_metric_ids, response_metrics_n),
        }

        response_dicts.append(response_dict)
        i += 1


responses = _map(lambda d: Response(**d), response_dicts)


In [8]:
for question in questions:
    api.create_question(question)

for form in forms:
    api.create_form(form)

for metric in metrics:
    api.create_metric(metric)

for response in responses:
    api.create_response(response)


## Testing

In [9]:
api.get_questions()
api.get_question("question-0")
api.get_forms()
api.get_form("form-1")
api.get_metrics()
api.get_metric("metric-0")
api.get_responses("form-1")
api.get_response("form-1", "response-10")


ResponseSnapshot(id='response-10', createdAt=0, updatedAt=0, formId='form-1', trackingId='tracking-1', answers={'question-9': 'Yes', 'question-2': 'Yes', 'question-0': 'No'}, metrics=[Metric(id='metric-5', createdAt=0, updatedAt=0, trackingId='tracking-0', values={'metric-value-0': 0.7358165650283597, 'metric-value-2': 0.2494995864507321, 'metric-value-1': 0.32882327934183064}), Metric(id='metric-3', createdAt=0, updatedAt=0, trackingId='tracking-0', values={'metric-value-0': 0.21221041390633455, 'metric-value-2': 0.14227552955777933, 'metric-value-1': 0.2885097260431845})])

In [10]:
api.get_form_metrics("form-1")
api.get_response_metrics("form-1", "response-10")
api.get_tracking("tracking-0")


[Metric(id='metric-0', createdAt=0, updatedAt=0, trackingId='tracking-0', values={'metric-value-0': 0.10098265357445102, 'metric-value-2': 0.8026990787260162, 'metric-value-1': 0.7632350071096584}),
 Metric(id='metric-1', createdAt=0, updatedAt=0, trackingId='tracking-0', values={'metric-value-0': 0.5980494089596754, 'metric-value-2': 0.1247390116310475, 'metric-value-1': 0.635543014576007}),
 Metric(id='metric-2', createdAt=0, updatedAt=0, trackingId='tracking-0', values={'metric-value-0': 0.1866218810592032, 'metric-value-2': 0.9503041284170758, 'metric-value-1': 0.42833032789280545}),
 Metric(id='metric-3', createdAt=0, updatedAt=0, trackingId='tracking-0', values={'metric-value-0': 0.21221041390633455, 'metric-value-2': 0.14227552955777933, 'metric-value-1': 0.2885097260431845}),
 Metric(id='metric-4', createdAt=0, updatedAt=0, trackingId='tracking-0', values={'metric-value-0': 0.8778234468875254, 'metric-value-2': 0.6842843169900534, 'metric-value-1': 0.32476995243259854}),
 Metri

In [11]:
api.get_response_analytics("form-1", "response-10")

ResponseAnalytics(id='response-10', createdAt=1679000159, updatedAt=1679000159, status='Finished', jobId='058c1ebb-f6c2-496e-878b-81a2a193e6d9', formId='form-1', trackingId='tracking-1', keywords={'question-3': [], 'question-4': [], 'question-6': []}, emotions={'question-3': [Emotion(label='realization', score=0.12365546822547913), Emotion(label='approval', score=0.09206066280603409), Emotion(label='caring', score=0.07140208780765533), Emotion(label='pride', score=0.05091802403330803), Emotion(label='curiosity', score=0.04539746791124344), Emotion(label='optimism', score=0.03649824112653732), Emotion(label='amusement', score=0.03512772172689438), Emotion(label='desire', score=0.034038905054330826), Emotion(label='neutral', score=0.03403535857796669), Emotion(label='excitement', score=0.03266259282827377), Emotion(label='joy', score=0.03238248452544212), Emotion(label='admiration', score=0.031381476670503616), Emotion(label='nervousness', score=0.03017023205757141), Emotion(label='relie

In [12]:
api.get_form_analytics("form-1")


FormAnalytics(id='form-1', createdAt=1679000280, updatedAt=1679000280, status='Finished', jobId='b2f9978f-3407-44b1-8969-653742628c43', formId='form-1', keywords={'question-3': [], 'question-4': [], 'question-6': []}, emotions={'question-3': [Emotion(label='caring', score=0.054907992109656335), Emotion(label='approval', score=0.0683763436973095), Emotion(label='disapproval', score=0.07574186101555824), Emotion(label='realization', score=0.0980410985648632), Emotion(label='remorse', score=0.030416952818632124), Emotion(label='desire', score=0.02876665610820055), Emotion(label='curiosity', score=0.036093606241047384), Emotion(label='excitement', score=0.02482333783991635), Emotion(label='pride', score=0.040793842263519765), Emotion(label='confusion', score=0.04054570719599724), Emotion(label='relief', score=0.027374593541026115), Emotion(label='admiration', score=0.025070648454129697), Emotion(label='surprise', score=0.021815712563693523), Emotion(label='neutral', score=0.036022301018238

In [13]:
question = Question(
    id="new-question",
    createdAt=0,
    updatedAt=0,
    type="ShortAnswer",
    required=True,
    question="What is your favorite color?",
    placeholder=None,
    subQuestions=None,
    options=None,
    min=None,
    max=None,
)
api.create_question(question)

api.add_question_to_form("form-0", "new-question")


In [14]:
metric = Metric(
    id="new-metric",
    createdAt=0,
    updatedAt=0,
    trackingId="tracking-0",
    values={"metric-value-0": 0.5, "metric-value-1": 0.5, "metric-value-2": 0.5},
)
api.create_metric(metric)


api.add_metric_to_response("form-0", "response-0", "new-metric")


In [15]:
api.patch_question(
    "new-question",
    {
        "question": "Is this a new question?",
    },
)
api.patch_form(
    "form-0",
    {
        "title": "New title",
    }
)
api.patch_metric(
    "new-metric",
    {
        "values": {"metric-value-0": 1., "metric-value-1": 1.0, "metric-value-2": 1.0},
    }
)
api.patch_response(
    "form-0",
    "response-0",
    {
        "updatedAt": 1,
    }
)


In [16]:
api.delete_form("form-0")
api.delete_response("form-0", "response-0")

In [17]:
api.set_form_active("form-1", False)

In [20]:
api.move_question_up("form-1", "question-0")

In [21]:
api.move_question_down("form-1", "question-0")